In [1]:
import json
import numpy as np
from bottom_up.all import pick_bfs_connected
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import get_local_feats
from bottom_up.all import get_global_feats
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import pick_l2r_connected
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import pick_bfs_connected
from bottom_up.all import len_tree
%load_ext autoreload

%autoreload 2

In [8]:
import pickle
with open("a", "rb") as inf:
    dt = pickle.load(inf)
    
clf, vectorizer = dt

/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [109]:
def pick_at_random(l, **kwargs):
    return random.sample(l, 1)[0]

from code.treeops import bfs
from bottom_up.all import pick_bfs
from bottom_up.all import get_dependents_and_governors
import random

def runtime_path_wild_frontier(sentence, pi, clf, vectorizer, verbose=False):
    T = {i for i in sentence["q"]}
    F = set()
    
    # init frontier
    for v in T:
        for i in sentence["tokens"]:
            F.add(i["index"])

    d, heads, c = bfs(g=sentence, hop_s=0) 
    while len(F) > 0:
        vertex = pi(F=F, d=d, T=T, s=sentence)

        if vertex != 0: # bug here?
            feats = get_local_feats(vertex=vertex, sentence=sentence, d=d, current_tree=T)
            feats = get_global_feats(sentence=sentence, feats=feats, vertex=vertex, current_tree=T)

            X = vectorizer.transform([feats])
            pred = clf.predict_proba(X)[0]
            y = clf.predict(X)[0]

            if y == 1:
                T.add(vertex)
                if vertex != 0:
                    for i in get_dependents_and_governors(vertex, sentence, T):
                        if i not in T and i is not None:
                            F.add(i)
                else:
                    for i in get_dependents(sentence, vertex):
                        if i["dependent"] not in T and i is not None:
                            F.add(i)
        F.remove(vertex)

    return T

tot = 0
for pno, paths in enumerate(validation_paths):
    paths=json.loads(paths)
    sentence = paths["sentence"]
    predicted = runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)
    f1s = get_f1(predicted, sentence)
    tot += f1s
print(tot/len(validation_paths))

0.8497676503318201


In [108]:
from bottom_up.all import get_labels_and_features
from bottom_up.all import get_governor

import json
import pickle
import numpy as np
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import len_tree

training_paths = [_ for _ in open("training.paths")]
validation_paths = [_ for _ in open("validation.paths")]


vectorizer = DictVectorizer(sparse=True)

train_features, train_labels = get_labels_and_features(training_paths)

X_train = vectorizer.fit_transform(train_features)

y_train = np.asarray(train_labels)

val_features, val_labels = get_labels_and_features(validation_paths)

X_val = vectorizer.transform(val_features)

y_val = np.asarray(val_labels)

clf = LogisticRegression(random_state=0,
                         solver='lbfgs',
                         C=.05,
                         multi_class='ovr').fit(X_train, y_train)

print(clf.score(X_val,y_val))
print(clf.score(X_train, y_train))

0.9446380484714082
0.95125013733461


In [92]:
confusion = []
from collections import Counter
from collections import defaultdict
all_vals = defaultdict(list)
scr = clf.predict(X_train)
o = 0
h = 0
tot = []
big_list = []
big_list_sentences = []

for paths in training_paths:
    paths=json.loads(paths)
    s = paths["sentence"]
    for p in paths["paths"]:
        current_tree, vertex, decision = p
        if vertex != 0:  
            big_list.append(p)
            big_list_sentences.append(s)
        

for ino, i in enumerate(train_features):
    if 'dep' in i and i["dep"] == "dobj":
        o += 1
        if scr[ino] == y_train[ino]:
            h += 1
        else:
            tot.append((scr[ino], y_train[ino], big_list_sentences[ino],train_features[ino], big_list[ino]))
print(h/(len(tot) + h))
print(len(tot))

0.8790860573715811
725


In [87]:
hits = 0
govs = defaultdict(int)
for t in tot[400:450]:
    pred, was, s, feats, path = t
    current_tree, vertex, decision = path
    current_tree.sort()
    governed = [_["dep"] for _ in s["basicDependencies"] if _["dependent"] == vertex]
    for g in governed:
        govs[g] += 1
    if g == "dobj":
        break
        
        
        
print(govs)
#print(hits)
#print(len(tot))
#print(sum(govs.values()))

defaultdict(<class 'int'>, {'dobj': 1})


In [88]:
print(current_tree, vertex, pred, was, feats)
pretty_print_conl(s, "basicDependencies")
print(s["compression_indexes"])

[1, 2, 3, 4, 5, 11] 9 1 0 {'dep': 'dobj', 'governorGloss': 'finished', 'dependentGloss': 'yards', 'type': 'CHILD', 'position': 0.34615384615384615, 'reportage_dobj': False, 'parent_label': 'dobjfinished', 'child_label': 'dobjyards', 'ner': 'O', 'pos': 'NNS', 'depth': 2, 'disconnected': 0, 'over_r': False, 'remaining': 0.6714285714285714}
1	Oakdale	<-compound-	students
2	Elementary	<-compound-	students
3	School	<-compound-	students
4	students	<-nsubj-	finished
5	finished	<-ROOT-	ROOT
6	the	<-det-	yards
7	last	<-amod-	yards
8	385	<-nummod-	yards
9	yards	<-dobj-	finished
10	,	<-punct-	yards
11	aka	<-appos-	yards
12	the	<-det-	.2
13	.2	<-dep-	aka
14	of	<-case-	miles
15	the	<-det-	miles
16	26.2	<-nummod-	miles
17	miles	<-nmod-	.2
18	,	<-punct-	.2
19	of	<-case-	marathon
20	the	<-det-	marathon
21	marathon	<-nmod-	.2
22	they	<-nsubj-	started
23	started	<-acl:relcl-	marathon
24	in	<-case-	April
25	April	<-nmod-	started
26	.	<-punct-	finished
[1, 2, 3, 4, 5, 11, 12, 13, 19, 20, 21]
